# **RAG Implementation**

On va commencer par installer les bibliothèques nécessaires

In [ ]:
!pip install langchain_community huggingface-hub transformers accelerate bitsandbytes pymupdf langchain faiss-cpu sentence-transformers


## **Initiation du LLM utilisé**

Insérez votre token accessible sur le site [Hugging Face](https://huggingface.co/settings/tokens)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

J'ai utilisé le modèle **bigscience/bloomz-7b1-mt**, vu qu'il est adapté au langage français, et qu'il ne nécessite pas beacoup de stockage.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "bigscience/bloomz-7b1-mt"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True
)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


In [ ]:
# Test de génération
prompt = ' c est quoi le nom de Einsten ?'
response = generator(prompt, max_new_tokens=400,  do_sample=True, temperature=0.1)
print(response[0]["generated_text"])

## **Files Upload**

Dans cette section on va uploader nos fichiers et construire les chunks et les indexer dans notre vectorstore.

In [ ]:
import fitz

def extract_text_by_page(file_path):
    doc = fitz.open(file_path)
    pages = []
    for i, page in enumerate(doc):
        text = page.get_text()
        if text.strip():  # ignorer les pages vides
            pages.append({"page": i + 1, "text": text})
    return pages

In [ ]:
text_HG = extract_text_by_page('/content/CollinsSuzanne-HungerGames-1HungerGames2008.French.ebook_1.pdf')

In [ ]:
text_HP1 = extract_text_by_page('/content/harry-potter-1-lecole-des-sorciers.pdf')

In [ ]:
text_HP2 = extract_text_by_page('/content/harry-potter-2-la-chambre-des-secrets.pdf')

## **Construction du vectorstore**

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

On implémente maintenant la fonction qui va créer nos chunks, il  contribue en même temps le nom du roman et le numéro de la page du chunk en tant que metadata.

In [ ]:
def split_pages_to_chunks(pages, book_title):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    all_chunks = []
    for page in pages:
        sub_docs = splitter.create_documents([page["text"]])
        for doc in sub_docs:
            doc.metadata["source"] = f'From {book_title} book, page {page["page"]}'

        all_chunks.extend(sub_docs)
    return all_chunks

In [ ]:
chunks = split_pages_to_chunks(text_HG, 'Hunger Games') + split_pages_to_chunks(text_HP1, 'Harry Potter, Book 1') + split_pages_to_chunks(text_HP2, 'Harry Potter, Book 2')

In [ ]:
print(chunks[0])

On peut maintenant construire notre vectorstore en utilisant l'indexation Faiss.

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)

## **Notre système RAG**

La dernière étape est de regrouper tout ce qui a precédé pour construire notre RAG.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

llm = HuggingFacePipeline(pipeline=generator)

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)



## **Questions tests**

Notre système est prêt maintenant à répondre aux questions des utilisateurs.

In [ ]:
def ask_question(query):
  result = qa_chain.run(query)
  print(result)

In [ ]:
query = "Pourquoi Hermione s'est enfermée dans les toilettes des filles ? "
ask_question(query)

In [ ]:
query = "À quelle heure le train Poudlard Express est parti de la gare ? "
ask_question(query)

In [ ]:
query = "Qui est Firenze et comment a-t-il aidé Harry ? "
ask_question(query)

In [ ]:
query = "Comment s'appelle l'ami roux de Harry ? "
ask_question(query)

In [ ]:
query = "Quel est le nom de la sœur de Katniss Everdeen dans Hunger Games ?  "
ask_question(query)

In [ ]:
query = "Quel est le point commun entre le père d'Harry Potter et le père du personnage de Hunger Games ? "
ask_question(query)

In [ ]:
query = "Qui est le plus âgé des personnages principaux entre Harry Potter et Katniss Everdeen ? "
ask_question(query)